In [2]:
!pip install -q numpy pandas scikit-learn joblib
!pip install -q qiskit qiskit-machine-learning qiskit-aer qiskit-algorithms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 7.4 MB/s eta 0:00:00


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import numpy as np
import pandas as pd
import joblib
import time
import warnings
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA

from qiskit.circuit.library import ZZFeatureMap, TwoLocal
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_algorithms.optimizers import COBYLA
from qiskit_aer import AerSimulator
from qiskit.primitives import BackendSampler

# Setup
warnings.filterwarnings('ignore')
os.makedirs("Result", exist_ok=True)
start_time = time.time()

# Load Dataset
df = pd.read_csv('/content/drive/My Drive/merged.csv')

# Select features
cols = ["Bwd Packet Length Std", "Flow Bytes/s", "Total Length of Fwd Packets", "Fwd Packet Length Std",
        "Flow IAT Std", "Flow IAT Min", "Fwd IAT Total", "Flow Duration", "Bwd Packet Length Max",
        "Flow IAT Max", "Flow IAT Mean", "Total Length of Bwd Packets", "Fwd Packet Length Min",
        "Bwd Packet Length Mean", "Flow Packets/s", "Fwd Packet Length Mean", "Total Backward Packets",
        "Fwd Packet Length Max", "Total Fwd Packets", "Bwd Packet Length Min", 'Label']

df = df[cols].fillna(0)
df['Label'] = df['Label'].apply(lambda x: 1 if x == "BENIGN" else 0)

# Limit size and balance dataset manually
benign_df = df[df['Label'] == 1].sample(n=400, random_state=42)
attack_df = df[df['Label'] == 0].sample(n=400, random_state=42)
balanced_df = pd.concat([benign_df, attack_df]).sample(frac=1, random_state=42)

x = balanced_df.drop("Label", axis=1)
y = balanced_df["Label"]

# Normalize
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

# Reduce dimensions
pca = PCA(n_components=4)
x_pca = pca.fit_transform(x_scaled)

# Small balanced subset for quick training
x_train, x_test, y_train, y_test = train_test_split(
    x_pca, y, train_size=0.8, stratify=y, random_state=42)

# Quantum Classifier Setup
num_qubits = x_pca.shape[1]
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=2)
ansatz = TwoLocal(num_qubits, ['ry', 'rz'], 'cz', reps=2)
optimizer = COBYLA(maxiter=100)

backend = AerSimulator()
sampler = BackendSampler(backend)

vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    sampler=sampler
)

print("Training VQC...")
vqc.fit(np.asarray(x_train), np.asarray(y_train))

# Predict and evaluate
y_pred = vqc.predict(np.asarray(x_test))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Save artifacts
joblib.dump(scaler, "Result/vqc_scaler.pkl")
joblib.dump(pca, "Result/vqc_pca_transform.pkl")
joblib.dump(cols[:-1], "Result/vqc_features_used.pkl")

print("\nTotal Time Taken: %.2f seconds" % (time.time() - start_time))


Training VQC...

Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.76      0.74        80
           1       0.75      0.70      0.72        80

    accuracy                           0.73       160
   macro avg       0.73      0.73      0.73       160
weighted avg       0.73      0.73      0.73       160


Total Time Taken: 507.47 seconds
